# Calibration of the Optimal Steps Model

This script is an attempt to recompute the results in silver-2016b page 51. The scenario [scenario](./../../../../Scenarios/ModelTests/TestOSM_calibration/rimea_04_calibration_osm.scenario) is based on the RiMEA-Test 4. We use the `Teleporter` to model a circular scenario and the parameter `useFreeSpaceOnly = false` to generate high densities. The following code plots all the necessary diagrams.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vadereanalysistool import ScenarioOutput, VadereProject
from scipy.optimize import curve_fit

sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.0})

def plotEvolution(vproject, ending, yval, ylabel):
    plots = []
    i = 1
    plt.figure(1, figsize=(30, 10))
    for outStr in project.output_dirs :
        out = project.output_dirs[outStr]
        dataFrames = []
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                #df = df[df.velocity > 0]
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['timeStep'] = df.timeStep.astype(int)
                ndf['scenario'] = [out.scenario['name']] * len(df.density)
                dataFrames.append(ndf)
            #concatFrames = pd.concat(dataFrames)
        plt.subplot(2, 1, i)
        plt.title(out.scenario['name'])
        plt.xlabel('timeStep')
        plt.ylabel(ylabel)
        sns.lineplot(x='timeStep', y=yval, data=pd.concat(dataFrames))
        i = i + 1

def plotFundamentalDiagram(vproject, ending):
    dataFrames = []
    for outStr in project.output_dirs :
        out = project.output_dirs[outStr]
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['scenario'] = [out.scenario['name']] * len(df.velocity)
                #ndf = ndf[ndf.density < 7]
                dataFrames.append(ndf)
    concatFrames = pd.concat(dataFrames)
    g = sns.relplot(x="density", y="velocity", hue="scenario", data=concatFrames,
                height=10, aspect=2)
    
def plotFundamentalDiagramScatter(vproject, ending, sep=False, width = 10, height = 5):
    dataFrames = []
    index = 0
    cols = 3
    rows = len(vproject.output_dirs) / cols + 1
    if not sep :
        fig, axs = plt.subplots(int(rows), int(cols), figsize=(height*rows, width*cols), sharex=False, sharey=True)       
    for outStr in vproject.output_dirs :
        out = vproject.output_dirs[outStr]
        for fileStr in out.files :
            if fileStr.endswith(ending) :
                if not sep : 
                    axes = axs[int(index / cols), int(index % cols)]
                else :
                    fig = plt.figure(1, figsize=(width, height))
                    axes = plt.axes()
                    fig.add_axes(axes)
                ndf = pd.DataFrame();
                df = out.files[fileStr]()
                ndf['density'] = df.density.astype(float)
                ndf['velocity'] = df.velocity.astype(float)
                ndf['scenario'] = [out.scenario['name']] * len(df.velocity)
                #ndf = ndf[ndf.density < 7]
                #plt.scatter()
                #axes.set_title(out.scenario['name'])
                axes.set_xlabel('density')
                axes.set_ylabel('velocity')
                axes.set_xticks([0,1,2,3,4,5,6])
                axes.set_yticks([0,0.5,1,1.5,2,2.5])
                axes.set_xlim(0,6)
                axes.set_ylim(0,2.5)
                axes.scatter(ndf['density'], ndf['velocity'], s=0.7, marker='*', color='#555555')
                wm = plotWeidmann(axes)
                popt, pcov = curve_fit(kladek, ndf['density'], ndf['velocity'], p0=(1.34, 1.913, 5.4))
                print(str(popt[0]) + "," + str(popt[1]) + "," + str(popt[2]))
                xx = np.linspace(0.1, 6, 1000)
                yy = kladek(xx, *popt)
                axes.plot(xx, yy, '--', c=sns.color_palette().as_hex()[1])     
                axes.legend(['Weidmann', 'regression', 'Simulated data'])
                index = index + 1;
                if sep :
                    fig.savefig("./"+out.scenario['name']+"_fundamental_diagram"+".png", bbox_inches='tight')
                    plt.show()
    if not sep :
        fig.savefig("./"+vproject.project_name+"_fundamental_diagrams"+".png", bbox_inches='tight')
        

In [ ]:
def plotWeidmann(axes):
    wmaxDensity = 5.4
    wmeanVelocity = 1.34
    wgamma = 1.913
    wx = np.linspace(0.1, wmaxDensity, 100)
    return plotKladek(wx, wmeanVelocity, wgamma, wmaxDensity, axes)

def plotKladek(x, v, gamma, pmax, axes):
    result, = axes.plot(x, kladek(x, v, gamma, pmax), c=sns.color_palette().as_hex()[0])
    return result

def kladek(x, v, gamma, pmax):
    return v * (1 - np.exp(-gamma * (1/x - 1/pmax)))

First we load the project

In [ ]:
projectFolder = "./../../../../Scenarios/ModelCalibration/TestOSM_calibration/"
project = VadereProject(projectFolder)
#out = project.named_output.C_050_180_180_2018_11_26_16_30_29_355()'

# Meassurement methods
All methods are described in zhang-2011.

## Method A Plots
The computation of the velocity is slightly different i.e. we use the velocity computed by the current and last foot step.

In [ ]:
# transform data frame
plotFundamentalDiagramScatter(project, "aTimeStep.fundamentalDiagram", True, 6, 6)

## Method B Plots
This method does not work for this scenario since agents run multiple times through the same measurement area.

## Method C Plots
The computation of the velocity is slightly different i.e. we use the velocity computed by the current and last foot step.

In [ ]:
plotFundamentalDiagramScatter(project, "cTimeStep.fundamentalDiagram", True, 6, 6)

## Method D Plots
The computation of the velocity is slightly different i.e. we use the velocity computed by the current and last foot step.

In [ ]:
myPlots = plotFundamentalDiagramScatter(project, "dTimeStep.fundamentalDiagram", True, 6, 6)

## Method E Plots
This method is similar to method D but the density is defined by $$\langle \rho \rangle = \frac{1}{N} \sum\limits_{i=1}^{N} A_i ,$$
and the velocity is defined by 
$$\langle v \rangle = \sum\limits_{i=1}^{N} \frac{1}{A_i} \left( \sum\limits_{i=1}^{N} A_i v_i(t) \right) ,$$

where $N$ is the number of pedestrians inside the measurement area and $A_i$ is the area of the voronoi cell of agent $i$.

In [ ]:
plotFundamentalDiagramScatter(project, "eTimeStep.fundamentalDiagram", False, 10, 10)

## Density and velocity evolution

In [ ]:
plotEvolution(project, "dTimeStep.fundamentalDiagram", "density", "density")
#plotEvolution(project, "aTimeStep.fundamentalDiagram", "density", "density")

In [ ]:
plotEvolution(project, "dTimeStep.fundamentalDiagram", "velocity", "velocity")

In [ ]:
sns.color_palette()